This model predicts the numbers via hand gestures using Deep Learning, CNN, OpenCV, Skimage, Tensorflow. 




In [ ]:
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Activation
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers

import os, glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

import pandas as pd
import skimage

%matplotlib inline
import os
from skimage.feature import hog
from skimage import io, transform
import time 
from cv2 import IMREAD_GRAYSCALE

In [ ]:
train_images = glob.glob("../input/fingers/train/*.png")
test_images = glob.glob("../input/fingers/test/*.png")

Load the train and test images for Deep Learning Model.

In [ ]:
X_train = []
X_test = []
y_train = []
y_test = []
    
for img in train_images:
    img_read = io.imread(img)

    img_read = transform.resize(img_read, (128,128), mode = 'constant')#resize images to make them of same size
    X_train.append(img_read)
    y_train.append(img[-6:-4])
    
for img in test_images:
    img_read = io.imread(img)
    img_read = transform.resize(img_read, (128,128), mode = 'constant')
    X_test.append(img_read)
    y_test.append(img[-6:-4])

In [ ]:
X_train = np.expand_dims(X_train, axis=3)
X_test = np.expand_dims(X_test, axis=3)

Classes for hand gestures.

In [ ]:
label_to_int={
    '0R' : 0,
    '1R' : 1,
    '2R' : 2,
    '3R' : 3,
    '4R' : 4,
    '5R' : 5,
    '0L' : 6,
    '1L' : 7,
    '2L' : 8,
    '3L' : 9,
    '4L' : 10,
    '5L' : 11
}

In [ ]:
temp = []
for label in y_train:
    temp.append(label_to_int[label])
y_train = temp.copy()

temp = []
for label in y_test:
    temp.append(label_to_int[label])
y_test = temp.copy()

y_train = keras.utils.to_categorical(y_train, num_classes = 12)
y_test = keras.utils.to_categorical(y_test, num_classes = 12)

Define the architecture of the Deep Learning model using Convolutional Layers.
Conv2D and MaxPooling layers are used from keras. The model used is Sequential. Finally the model is complied to train it in next step.

In [ ]:
weight_decay = 1e-4

num_classes = 12

model = Sequential()

model.add(Conv2D(64, (4,4), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=(128,128,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())

model.add(Conv2D(64, (4,4), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2)) #Dropout layer is added to prevent overfitting
 
model.add(Conv2D(128, (4,4), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (4,4), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (4,4), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (4,4), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(128, activation="linear"))
model.add(Activation('elu'))
model.add(Dense(num_classes, activation='softmax'))
#Now compile the model using appropriate loss function and optimizer. 
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(0.0003), metrics=['accuracy'])
 
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 128, 128, 64)      1088      
_________________________________________________________________
activation_7 (Activation)    (None, 128, 128, 64)      0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 128, 128, 64)      256       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 128, 128, 64)      65600     
_________________________________________________________________
activation_8 (Activation)    (None, 128, 128, 64)      0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 128, 128, 64)      256       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 64, 64, 64)       

Now fit the train data into compiled model. 

In [ ]:
model.fit(x = X_train,y = y_train, batch_size=64, validation_data = (X_test,y_test), epochs = 5)

Epoch 1/5
282/282 [==============================] - 58s 192ms/step - loss: 1.7166 - accuracy: 0.7508 - val_loss: 19.2836 - val_accuracy: 0.0833
Epoch 2/5
282/282 [==============================] - 53s 188ms/step - loss: 0.0571 - accuracy: 0.9993 - val_loss: 0.8843 - val_accuracy: 0.7794
Epoch 3/5
282/282 [==============================] - 53s 188ms/step - loss: 0.0540 - accuracy: 1.0000 - val_loss: 0.0547 - val_accuracy: 0.9997
Epoch 4/5
282/282 [==============================] - 53s 188ms/step - loss: 0.0528 - accuracy: 1.0000 - val_loss: 0.0523 - val_accuracy: 1.0000
Epoch 5/5
282/282 [==============================] - 53s 188ms/step - loss: 0.0519 - accuracy: 1.0000 - val_loss: 0.0511 - val_accuracy: 1.0000


Woah!. We achieved accuracy of 100% :)


In [ ]:
model.save('/content/CV_model.h5')

In [ ]:
from keras.models import load_model

model = load_model('/content/CV_model.h5') #Load the trained model for use in CV

**capture_image** function takes the image of user after 3sec countdown using camera if available.

In [ ]:
def capture_image():
    """
    returns: image after converting into proper format
    Suggestion: Function can be modified to provide live feedback also
    """
    vid_cap = cv2.VideoCapture(0)
    captured_image = None
    timer = time.perf_counter()

    if not vid_cap.isOpened():
        print("Cannot open camera. Exiting..")
        return None
    while True:
        ret, frame = vid_cap.read()
        if not ret:
            print("Cannot receive frame. Exiting..")
            break
        plt.imshow(frame)
        captured_image = frame
        #captured_image = cv2.cvtColor(captured_image, cv2.COLOR_BGR2GRAY)
        if cv2.waitKey(1) == ord('q') or time.perf_counter()- timer > 3:
            break
    #avg_frame = cv2.boxFilter(captured_image,1,(10,10))
    #captured_image = captured_image - avg_frame
    vid_cap.release()
    cv2.destroyAllWindows()
    return captured_image



Previously attempted approach for this model without DL.

In [ ]:
# def process_and_load_template(image_path, pixelpercell, img_shape):
#     """
#         temp: raw temp as a numpy array
#         returns : processes images to get avg images and finally returns hog image and its features
#     """
#     image_feature_list=[]
#     hog_image_list=[]
#     roots=[]
#     filenames=[]
#     for root, _, filename in os.walk(image_path):
#       if len(filename)>0:
#         roots.append(root)
#         filenames.append(filename)

#     for i in range(1,6):
#       function='finger'+'_avg'
#       a=eval(function+'(filenames[i-1],img_shape,roots[i-1])')
#       # for i in range(0, a.shape[0]):
#       #   for j in range(0, a.shape[1]):
#       #     if a[i,j] < 150:
#       #       a[i,j] = a[i,j]/2
#       #     else:
#       #       a[i,j] = 200
#       plt.imshow(a)
#       plt.show()
#       (feature, hog_image)=hog(a,pixels_per_cell=(pixelpercell,pixelpercell),visualize=True)
#       image_feature_list.append(feature)
#       hog_image_list.append(hog_image)
#     return hog_image_list,image_feature_list


# def finger_avg(filename,img_shape,root):
   
#     avg_1_finger = np.zeros(img_shape)
    
#     for file_ in filename:
#         image = cv2.imread(os.path.join(root, file_),flags=IMREAD_GRAYSCALE)
#         image=cv2.resize(image,(img_shape[1],img_shape[0]))
#         avg_1_finger = np.asarray(image)+np.asarray(avg_1_finger)
    
        
#     avg_1_finger = avg_1_finger / len(filename)
#     return avg_1_finger

**load_data** function to load test images from dir if capture_image does not work.

In [ ]:
def load_data(data_dir):
    """
        Load the data to be predicted.
        Returns: tuple (images, counts)
        images: list of images as numpy array
        counts: list of finger count for each corresponding image
    """
    images = []
    counts = []
    for root, _, filenames in os.walk(data_dir):
        print(filenames)
        for file_ in filenames:
            image = cv2.imread(os.path.join(root, file_),flags=IMREAD_GRAYSCALE)
            images.append(image)
            count = os.path.join(root,file_).split("/").pop(-2)
            counts.append(count)

    
    return (images, counts)



**pre_process_image** manipulates the input image to extract important features and remove noise.

In [ ]:
def pre_process_image(img,scale = 0.9,minH = 100):
    """
        Manipulate the image accordingly
        making pyramids of images with different sizes
        Returns: processed_image (pyramid)
    """
    img=cv2.resize(img,(600,500))
    fil = cv2.GaussianBlur(img,(5,5),0)
    fil = (fil-fil.mean())/fil.std()   # normalisation
    pyramid = []   
    dst = np.copy(fil)
    pyramid = [dst]
    while dst.shape[0]>minH:
        dst = cv2.resize(dst,(int(dst.shape[0]*scale),int(dst.shape[1]*scale)))
        pyramid.append(dst)
    return pyramid

In [ ]:
# def sliding_win(img, hand_template, winH, winW,stepSize, pixels_per_cell):
#     #sliding window to detect location of hands
#     # and to make a box
#     """
#     Img: user image after processing
#     template: hand template to detect approx pos of hand
#     Returns : return co-ordinates of top_left point 
#     """
#     H,W=img.shape
#     pad_img = np.lib.pad(img,((winH // 2,winH - winH // 2),(winW // 2,winW - winW // 2)),mode='constant')
#     response_map = np.zeros((H // stepSize + 1, W // stepSize + 1))
#     maxscore=0

#     for i in range(0,H+1,stepSize):
#       for j in range(0,W+1,stepSize):
#         win=pad_img[i:i+winH,j:j+winW]
#         hogfeature=hog(win,pixels_per_cell=(pixels_per_cell,pixels_per_cell))
#         hogscore=np.dot(hogfeature,hand_template)
#         response_map[i//stepSize][j//stepSize]=hogscore
#         if hogscore> maxscore:
#           maxscore=hogscore
#           maxr=i-winH//2
#           maxc=j -winW//2
#     response_map = cv2.resize(response_map,(img.shape[1],img.shape[0]))
#     return (maxscore, maxr, maxc, response_map)



**sliding_win** slides a fixed size window over the input image and the window is sent to DL model to predict the hand gesture in that window. If no hand is present that window is discarded as we used threshold value on probabilty to filter them.

In [ ]:
def sliding_win(img, winH, winW,stepSize):
    #sliding window to detect location of hands
    # and to make a box
    """
    Img: user image after processing
    template: hand template to detect approx pos of hand
    Returns : return co-ordinates of top_left point 
    """
    H,W=img.shape
    pad_img = np.lib.pad(img,((winH // 2,winH - winH // 2),(winW // 2,winW - winW // 2)),mode='constant')
    
    threshold=0.999
    maxr=[]
    maxc=[]
    values=[]
    maxscore=0
    for i in range(0,H+1,stepSize):
      for j in range(0,W+1,stepSize):
        win=pad_img[i:i+winH,j:j+winW]
        plt.imshow(win)
        plt.show()
        win = win.reshape(-1, 128, 128, 1)
        score=[]
        score=model.predict(win)        
        n=np.argmax(score)
        if score[0,n]>threshold :
          maxscore=score[0,n]
          values.append(n)
          maxr.append(i-WinH//2)
          maxc.append(j-WinW//2)
  
    return (values,maxr, maxc,maxscore)

**display_hand** displays the box around detected hand for visual and debugging purpose.

In [ ]:

def display_hand(img, winH, winW, top_left_point):
    # window output
    """
    returns: nothing, displays a window around selected hand
    """
    (H,W)=top_left_point
    boxed_img=cv2.rectangle(img,(W,H),(W+winW,H+winH),(0, 0, 0),2)
    plt.imshow(boxed_img)
    plt.show()

In [ ]:
# def template_match(img, template, topleft_x, topleft_y, win_H, win_W, pix_per_cell):
#     # Match the template with 5 hand templates
#     """
#     img: user image
#     template: matching template
#     returns: correlation score
#     """
#     hog_feature = hog(img[topleft_y:topleft_y+win_H,topleft_x:topleft_x+win_W],pixels_per_cell=(pix_per_cell, pix_per_cell))
#     hog_score = np.dot(hog_feature, template)
    
#     return hog_score


In [ ]:
WinH=128
WinW=128
stepSize=32
#pixels_per_cell=2

# img_dir_path='/Users/hp/OneDrive/Desktop/data_images/image_data'
# (images, labels) = load_data(img_dir_path)
# a, templates = process_and_load_template(img_dir_path,pixels_per_cell,(WinH,WinW))
# for i in range (5):
#     plt.imshow(a[i])
#     plt.show()

**This code controls the flow of the whole model and returns the predicted hand gesture number.**

In [ ]:
img=capture_image()

plt.imshow(img)
plt.show()

img=transform.resize(img, (500,600), mode = 'constant')

current_scale=10/9
score = 0
max_score=0
scale=0
rows=[]
cols=[]
final_rows=[]
final_cols=[]
values=[]
final_values=[]
count=0
finger_count = 0

#loop over all the images of different scales in image pyramid 
pyramid = pre_process_image(img,0.9,300)

for i in pyramid:
    # plt.imshow(i)
    # plt.show()

    current_scale=current_scale*0.9
    values,rows,cols,score=sliding_win(i,WinH,WinW,stepSize)

    if score>max_score:
      max_score=score
      final_rows=rows
      final_col=cols
      final_values=values
      scale=current_scale

img=transform.resize(img, (int(500*scale),int(600*scale)), mode = 'constant')
for i,j in zip(final_rows, final_cols):
    display_hand(img,WinH,WinW,(i,j))

print("Predicted finger count: ",final_values)


 
# #Display the detected hand for debugging 
# display_hand(img, WinH, WinW, (finalr, finalc))
# #Find the best matching template
# for t in templates:
#     count+=1
#     temp_score = template_match(img,t,finalr, finalc,WinH, WinW, 8)
#     print(temp_score)
#     if temp_score > max_temp_score:
#         max_temp_score = temp_score
#         finger_count = count
#         print(count)

# print("Predicted finger count: ",finger_count)
# #print("Actual answer: ", label)
# print("##########################")